In [0]:
#Kaggle link: https://www.kaggle.com/c/ashrae-energy-prediction
import zipfile
import pandas as pd
with zipfile.ZipFile("/content/sample_data/ashrae-energy-prediction-final-test.zip") as z:
    with z.open("ashrae-energy-prediction/building_metadata.csv") as f:
        building_meta = pd.read_csv(f)
    with z.open("ashrae-energy-prediction/train.csv") as f:
        train_data = pd.read_csv(f)
    with z.open("ashrae-energy-prediction/weather_train.csv") as f:
        weather_train = pd.read_csv(f)
    with z.open("ashrae-energy-prediction/weather_test.csv") as f:
        weather_test = pd.read_csv(f)
    with z.open("ashrae-energy-prediction/test.csv") as f:
        test_data = pd.read_csv(f)

In [2]:
weather_train.isnull().sum()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64

In [0]:
weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp'])

In [0]:
for site in weather_train['site_id'].unique():
  #site_weather = weather_train[weather_train['site_id'] == site]
  weather_train[weather_train['site_id'] == site] = weather_train[weather_train['site_id'] == site].interpolate(method="linear")

In [0]:
weather_train_interpolated = pd.DataFrame([])

In [0]:
startDate = pd.to_datetime("2016-01-01 00:00:00")
endDate = pd.to_datetime("2016-12-31 23:00:00")

In [0]:
for site in weather_train['site_id'].unique():
  site_weather = weather_train[weather_train['site_id'] == site]
  site_weather = site_weather.set_index('timestamp')
  site_weather_reindexed = site_weather.resample("1H")
  #new_index = pd.date_range(startDate,endDate,freq="1H")
  #site_weather_reindexed = site_weather.reindex(new_index)
  site_weather_reindexed = site_weather_reindexed.interpolate(method="linear")
  weather_train_interpolated = weather_train_interpolated.append(site_weather_reindexed)

In [0]:
weather_train_interpolated = weather_train_interpolated.reset_index()

In [9]:
weather_train_interpolated.isnull().sum()

timestamp                 0
site_id                   0
air_temperature           0
cloud_coverage        17606
dew_temperature           0
precip_depth_1_hr     26568
sea_level_pressure     8970
wind_direction            0
wind_speed                0
dtype: int64

In [0]:
#Filling missing values for sea_level_pressure with mean and precip_depth_1_hr, cloud_coverage with mode
weather_train_interpolated.loc[(weather_train_interpolated['sea_level_pressure'].isnull()), 'sea_level_pressure'] = weather_train_interpolated['sea_level_pressure'].mean()
weather_train_interpolated.loc[(weather_train_interpolated['precip_depth_1_hr'].isnull()), 'precip_depth_1_hr'] = 0.0
weather_train_interpolated.loc[(weather_train_interpolated['cloud_coverage'].isnull()), 'cloud_coverage'] = 0.0

In [11]:
weather_train_interpolated.isnull().sum()

timestamp             0
site_id               0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [12]:
building_meta.isnull().sum()

site_id           0
building_id       0
primary_use       0
square_feet       0
year_built      774
floor_count    1094
dtype: int64

In [0]:
building_meta.loc[(building_meta['floor_count'].isnull()), 'floor_count'] = 1
building_meta.loc[(building_meta['year_built'].isnull()), 'year_built'] = 1976.0

In [0]:
#Creating new feature called total area based on floor count and square feet
building_meta['total_area'] = building_meta['floor_count'] * building_meta['square_feet']

In [0]:
#Dropping square feet and floor count column
building_meta.drop(columns=['square_feet','floor_count'],inplace=True)

In [16]:
train_data.isnull().sum()

building_id      0
meter            0
timestamp        0
meter_reading    0
dtype: int64

In [0]:
building_meta = pd.get_dummies(building_meta, columns=['primary_use'])

In [0]:
merged_train_building = pd.merge(train_data, building_meta, on='building_id')

In [19]:
merged_train_building.head()

,building_id,meter,timestamp,meter_reading,site_id,year_built,total_area,primary_use_Education,primary_use_Entertainment/public assembly,primary_use_Food sales and service,primary_use_Healthcare,primary_use_Lodging/residential,primary_use_Manufacturing/industrial,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage
0,0,0,2016-01-01 00:00:00,0.0,0,2008.0,7432.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,2016-01-01 01:00:00,0.0,0,2008.0,7432.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,2016-01-01 02:00:00,0.0,0,2008.0,7432.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,2016-01-01 03:00:00,0.0,0,2008.0,7432.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,2016-01-01 04:00:00,0.0,0,2008.0,7432.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#merged_train_building.dtypes
merged_train_building['timestamp'] = pd.to_datetime(merged_train_building['timestamp'])

In [21]:
merged_train_building.head()

,building_id,meter,timestamp,meter_reading,site_id,year_built,total_area,primary_use_Education,primary_use_Entertainment/public assembly,primary_use_Food sales and service,primary_use_Healthcare,primary_use_Lodging/residential,primary_use_Manufacturing/industrial,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage
0,0,0,2016-01-01 00:00:00,0.0,0,2008.0,7432.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,2016-01-01 01:00:00,0.0,0,2008.0,7432.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,2016-01-01 02:00:00,0.0,0,2008.0,7432.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,2016-01-01 03:00:00,0.0,0,2008.0,7432.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,2016-01-01 04:00:00,0.0,0,2008.0,7432.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
weather_train_interpolated.head()

,timestamp,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,2016-01-01 00:00:00,0.0,25.0,6.0,20.0,0.0,1019.7,0.0,0.0
1,2016-01-01 01:00:00,0.0,24.4,4.0,21.1,-1.0,1020.2,70.0,1.5
2,2016-01-01 02:00:00,0.0,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,2016-01-01 03:00:00,0.0,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,2016-01-01 04:00:00,0.0,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [23]:
merged_train_building.shape

(20216100, 23)

In [24]:
weather_train_interpolated.shape

(140537, 9)

In [0]:
train = pd.merge(merged_train_building, weather_train_interpolated, on=['site_id','timestamp'])

In [0]:
site_zero_buildings = train[train['site_id'] == 0]['building_id'].values.flatten().tolist()

In [0]:
kbtu_conversion_factor = 0.293071
train.loc[(train['building_id'].isin(site_zero_buildings)) & (train['meter'] == 0),'meter_reading'] = train.loc[(train['building_id'].isin(site_zero_buildings)) & (train['meter'] == 0),'meter_reading']*kbtu_conversion_factor

In [0]:
train['weekday'] = train['timestamp'].dt.weekday

In [0]:
train['weekday'] = train['weekday'].isin([0,1,2,3,4])

In [0]:
#Extracting hour from timestamp
train['hour'] = train['timestamp'].dt.hour

In [0]:
#Extracting month from timestamp
train['month'] = train['timestamp'].dt.month

In [0]:
#Extracting year from timestamp
train['year'] = train['timestamp'].dt.year

In [0]:
#Creating new feature called building age based on this previous year and the year which building was build
train['building_age'] = train['year'] - train['year_built']

In [0]:
#Drop unwanted columns related to year
#train = train.drop(columns=['year_built', 'year'])
del train['year_built']
del train['year']

**Data Normalization**

In [35]:
from sklearn.preprocessing import PowerTransformer
at_pt = PowerTransformer(method='yeo-johnson')
train['air_temperature'] = at_pt.fit_transform(train[['air_temperature']])
train['air_temperature'].skew()

-0.15778677535064728

In [36]:
cc_pt = PowerTransformer(method='yeo-johnson')
train['cloud_coverage'] = cc_pt.fit_transform(train[['cloud_coverage']])
train['cloud_coverage'].skew()

-0.003063201517380995

In [37]:
dew_pt = PowerTransformer(method='yeo-johnson')
train['dew_temperature'] = dew_pt.fit_transform(train[['dew_temperature']])
print(train['dew_temperature'].skew())

-0.22318298822822247


In [38]:
pre_pt = PowerTransformer(method='yeo-johnson')
train['precip_depth_1_hr'] = pre_pt.fit_transform(train[['precip_depth_1_hr']])
print(train['precip_depth_1_hr'].skew())

-0.6175721293720853


In [39]:
ws_pt = PowerTransformer(method='yeo-johnson')
train['wind_speed'] = ws_pt.fit_transform(train[['wind_speed']])
print(train['wind_speed'].skew())

-0.021589399707228635


In [40]:
ta_pt = PowerTransformer(method='yeo-johnson')
train['total_area'] = ta_pt.fit_transform(train[['total_area']])
print(train['total_area'].skew())

0.01976005042213119


In [41]:
ba_pt = PowerTransformer(method='yeo-johnson')
train['building_age'] = ba_pt.fit_transform(train[['building_age']])
print(train['building_age'].skew())

0.24334356637168034


In [42]:
mr_pt = PowerTransformer(method='yeo-johnson')
train['meter_reading'] = mr_pt.fit_transform(train[['meter_reading']])
print(train['meter_reading'].skew())

0.0025713174838154786


In [43]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,total_area,primary_use_Education,primary_use_Entertainment/public assembly,primary_use_Food sales and service,primary_use_Healthcare,primary_use_Lodging/residential,primary_use_Manufacturing/industrial,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,weekday,hour,month,building_age
0,0,0,2016-01-01,-1.825863,0,-1.691510,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.834089,1.151061,1.249691,0.067746,1019.7,0.0,-1.847361,True,0,1,-2.177341
1,1,0,2016-01-01,-1.825863,0,-2.384460,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.834089,1.151061,1.249691,0.067746,1019.7,0.0,-1.847361,True,0,1,-1.837914
2,2,0,2016-01-01,-1.825863,0,-1.916981,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.834089,1.151061,1.249691,0.067746,1019.7,0.0,-1.847361,True,0,1,-0.933660
3,3,0,2016-01-01,-1.825863,0,-0.867037,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.834089,1.151061,1.249691,0.067746,1019.7,0.0,-1.847361,True,0,1,-1.682631
4,4,0,2016-01-01,-1.825863,0,0.312806,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.834089,1.151061,1.249691,0.067746,1019.7,0.0,-1.847361,True,0,1,-0.026442


In [0]:
#One hot encoding
#train = pd.get_dummies(train, columns=['primary_use'])

In [45]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,total_area,primary_use_Education,primary_use_Entertainment/public assembly,primary_use_Food sales and service,primary_use_Healthcare,primary_use_Lodging/residential,primary_use_Manufacturing/industrial,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,weekday,hour,month,building_age
0,0,0,2016-01-01,-1.825863,0,-1.691510,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.834089,1.151061,1.249691,0.067746,1019.7,0.0,-1.847361,True,0,1,-2.177341
1,1,0,2016-01-01,-1.825863,0,-2.384460,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.834089,1.151061,1.249691,0.067746,1019.7,0.0,-1.847361,True,0,1,-1.837914
2,2,0,2016-01-01,-1.825863,0,-1.916981,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.834089,1.151061,1.249691,0.067746,1019.7,0.0,-1.847361,True,0,1,-0.933660
3,3,0,2016-01-01,-1.825863,0,-0.867037,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.834089,1.151061,1.249691,0.067746,1019.7,0.0,-1.847361,True,0,1,-1.682631
4,4,0,2016-01-01,-1.825863,0,0.312806,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.834089,1.151061,1.249691,0.067746,1019.7,0.0,-1.847361,True,0,1,-0.026442


In [0]:
target = train['meter_reading']
del train['meter_reading']

In [47]:
#Splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.05, random_state=9)
print(X_train.shape)
print(X_test.shape)

(19204236, 32)
(1010750, 32)


In [0]:
def df_clean(df):
 cols = ['meter', 'total_area', 'air_temperature', 'cloud_coverage', 'dew_temperature',  'precip_depth_1_hr',  'sea_level_pressure', 'wind_direction', 'wind_speed', 'weekday',  'hour', 'month',  'building_age', 'primary_use_Education',  'primary_use_Entertainment/public assembly',  'primary_use_Food sales and service', 'primary_use_Healthcare', 'primary_use_Lodging/residential',  'primary_use_Manufacturing/industrial', 'primary_use_Office', 'primary_use_Other',  'primary_use_Parking',  'primary_use_Public services',  'primary_use_Religious worship',  'primary_use_Retail', 'primary_use_Services', 'primary_use_Technology/science', 'primary_use_Utility',  'primary_use_Warehouse/storage']
 #df_copy = df.copy()
 #df_copy = df_copy[cols]
 return df[cols]

In [0]:
X_train_model = df_clean(X_train)
X_test_model = df_clean(X_test)

In [0]:
def rmsle(expected, predicted):
   return np.sqrt(np.mean(np.square(np.log(predicted + 1) - np.log(expected + 1))))

In [51]:
#Training - Linear Regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train_model, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [52]:
train_pred = lr.predict(X_train_model)
X_train['meter_reading_predicted'] = train_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [53]:
X_train['meter_reading_predicted'] = mr_pt.inverse_transform(X_train['meter_reading_predicted'].values.reshape([-1,1]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
X_train.head()

,building_id,meter,timestamp,site_id,total_area,primary_use_Education,primary_use_Entertainment/public assembly,primary_use_Food sales and service,primary_use_Healthcare,primary_use_Lodging/residential,primary_use_Manufacturing/industrial,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,weekday,hour,month,building_age,meter_reading_predicted
10267804,893,0,2016-03-24 03:00:00,9,-0.363424,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.409726,0.750051,1.066941,0.067746,1006.7,150.000000,0.013163,True,3,3,-0.078986,44.078299
11133891,921,1,2016-07-20 14:00:00,9,0.933580,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.155563,0.143020,1.671798,0.067746,1020.8,46.666667,-0.791331,True,14,7,-0.078986,171.992724
5763855,534,0,2016-09-05 12:00:00,3,-3.125230,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.294940,0.143020,0.651015,0.067746,1024.0,350.000000,0.230373,True,12,9,-0.078986,2.331659
12065703,945,1,2016-11-24 17:00:00,9,1.118341,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.294940,-1.155279,0.188518,0.067746,1024.1,0.000000,-0.791331,True,17,11,-0.078986,226.880599
3310107,201,3,2016-08-31 03:00:00,2,0.796624,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.924617,0.750051,0.534647,0.067746,1006.0,90.000000,0.631694,True,3,8,-0.996242,175.181249


In [55]:
X_train.loc[(X_train['building_id'].isin(site_zero_buildings)) & (X_train['meter'] == 0),'meter_reading_predicted'] = X_train.loc[(X_train['building_id'].isin(site_zero_buildings)) & (X_train['meter'] == 0),'meter_reading_predicted']/kbtu_conversion_factor

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [56]:
X_train['meter_reading'] = mr_pt.inverse_transform(y_train.values.reshape([-1,1]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [57]:
X_train.loc[(X_train['building_id'].isin(site_zero_buildings)) & (X_train['meter'] == 0),'meter_reading'] = X_train.loc[(X_train['building_id'].isin(site_zero_buildings)) & (X_train['meter'] == 0),'meter_reading']/kbtu_conversion_factor

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [58]:
import numpy as np
print('RMSLE:', rmsle(X_train['meter_reading'], X_train['meter_reading_predicted']))

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


RMSLE: 1.9763884298847405


In [59]:
train_pred = lr.predict(X_test_model)
X_test['meter_reading_predicted'] = train_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [60]:
X_test['meter_reading_predicted'] = mr_pt.inverse_transform(X_test['meter_reading_predicted'].values.reshape([-1,1]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
X_test.loc[(X_test['building_id'].isin(site_zero_buildings)) & (X_test['meter'] == 0),'meter_reading_predicted'] = X_test.loc[(X_test['building_id'].isin(site_zero_buildings)) & (X_test['meter'] == 0),'meter_reading_predicted']/kbtu_conversion_factor

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [62]:
X_test['meter_reading'] = mr_pt.inverse_transform(y_test.values.reshape([-1,1]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
X_test.loc[(X_test['building_id'].isin(site_zero_buildings)) & (X_test['meter'] == 0),'meter_reading'] = X_test.loc[(X_test['building_id'].isin(site_zero_buildings)) & (X_test['meter'] == 0),'meter_reading']/kbtu_conversion_factor

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [64]:
print('Test RMSLE:', rmsle(X_test['meter_reading'], X_test['meter_reading_predicted']))

Test RMSLE: 1.9781520554203855


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [65]:
import joblib
joblib.dump([lr,at_pt,cc_pt,dew_pt,pre_pt,ws_pt,ta_pt,ba_pt,mr_pt],'lr_model.pkl')

['lr_model.pkl']

In [66]:
test_data.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00


In [67]:
weather_test.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6


In [0]:
weather_test['timestamp'] = pd.to_datetime(weather_test['timestamp'])
test_data['timestamp'] = pd.to_datetime(test_data['timestamp'])

In [0]:
for site in weather_test['site_id'].unique():
  weather_test[weather_test['site_id'] == site] = weather_test[weather_test['site_id'] == site].interpolate(method="linear")

In [70]:
weather_test.isnull().sum()

site_id                   0
timestamp                 0
air_temperature           0
cloud_coverage        33221
dew_temperature           0
precip_depth_1_hr     51812
sea_level_pressure    17241
wind_direction            0
wind_speed                0
dtype: int64

In [0]:
#Filling missing values for sea_level_pressure with mean and precip_depth_1_hr, cloud_coverage with mode
weather_test.loc[(weather_test['sea_level_pressure'].isnull()), 'sea_level_pressure'] = weather_test['sea_level_pressure'].mean()
weather_test.loc[(weather_test['precip_depth_1_hr'].isnull()), 'precip_depth_1_hr'] = 0.0
weather_test.loc[(weather_test['cloud_coverage'].isnull()), 'cloud_coverage'] = 0.0

In [72]:
test_data.shape

(41697600, 4)

In [0]:
merged_test_building = pd.merge(test_data, building_meta, on='building_id')

In [74]:
merged_test_building.shape

(41697600, 23)

In [0]:
test_final = pd.merge(merged_test_building, weather_test, on=['site_id','timestamp'],how="left")

In [76]:
test_final.shape

(41697600, 30)

In [77]:
test_final.isnull().sum()

row_id                                            0
building_id                                       0
meter                                             0
timestamp                                         0
site_id                                           0
year_built                                        0
total_area                                        0
primary_use_Education                             0
primary_use_Entertainment/public assembly         0
primary_use_Food sales and service                0
primary_use_Healthcare                            0
primary_use_Lodging/residential                   0
primary_use_Manufacturing/industrial              0
primary_use_Office                                0
primary_use_Other                                 0
primary_use_Parking                               0
primary_use_Public services                       0
primary_use_Religious worship                     0
primary_use_Retail                                0
primary_use_

In [0]:
test_final.loc[(test_final['air_temperature'].isnull()), 'air_temperature'] = test_final['air_temperature'].mean()
test_final.loc[(test_final['cloud_coverage'].isnull()), 'cloud_coverage'] = test_final['cloud_coverage'].mean()
test_final.loc[(test_final['dew_temperature'].isnull()), 'dew_temperature'] = test_final['dew_temperature'].mean()
test_final.loc[(test_final['precip_depth_1_hr'].isnull()), 'precip_depth_1_hr'] = 0
test_final.loc[(test_final['sea_level_pressure'].isnull()), 'sea_level_pressure'] = test_final['sea_level_pressure'].mean()
test_final.loc[(test_final['wind_direction'].isnull()), 'wind_direction'] = test_final['wind_direction'].mean()
test_final.loc[(test_final['wind_speed'].isnull()), 'wind_speed'] = test_final['wind_speed'].mean()

In [0]:
test_final['weekday'] = test_final['timestamp'].dt.weekday

In [0]:
test_final['weekday'] = test_final['weekday'].isin([0,1,2,3,4])

In [0]:
test_final['hour'] = test_final['timestamp'].dt.hour

In [0]:
test_final['month'] = test_final['timestamp'].dt.month

In [0]:
test_final['year'] = test_final['timestamp'].dt.year

In [0]:
test_final['building_age'] = test_final['year'] - test_final['year_built']

In [0]:
del test_final['year_built']
del test_final['year']

In [0]:
test_final['air_temperature'] = at_pt.transform(test_final[['air_temperature']])

In [0]:
test_final['cloud_coverage'] = cc_pt.transform(test_final[['cloud_coverage']])

In [0]:
test_final['dew_temperature'] = dew_pt.transform(test_final[['dew_temperature']])

In [0]:
test_final['precip_depth_1_hr'] = pre_pt.transform(test_final[['precip_depth_1_hr']])

In [0]:
test_final['wind_speed'] = ws_pt.transform(test_final[['wind_speed']])

In [0]:
test_final['total_area'] = ta_pt.transform(test_final[['total_area']])

In [0]:
test_final['building_age'] = ba_pt.transform(test_final[['building_age']])

In [0]:
x_test_final_model = df_clean(test_final)

In [0]:
#test_final_pred = lr.predict(x_test_final_model)
#test_final['meter_reading'] = test_final_pred

In [0]:
small_count = int(len(x_test_final_model)/10)
test_final_latest = pd.DataFrame([])
for i in range(10):
  #test_final_pred = lr.predict(x_test_final_model[small_count*i:small_count*(i+1)])
  test_final_latest = test_final_latest.append(pd.DataFrame(lr.predict(x_test_final_model[small_count*i:small_count*(i+1)])))

In [0]:
test_final['meter_reading'] = test_final_latest.values.flatten()

In [0]:
test_final['meter_reading'] = mr_pt.inverse_transform(test_final['meter_reading'].values.reshape([-1,1]))

In [0]:
#test_final.loc[(test_final['building_id'].isin(site_zero_buildings)) & (test_final['meter'] == 0),'meter_reading'] = test_final.loc[(test_final['building_id'].isin(site_zero_buildings)) & (test_final['meter'] == 0),'meter_reading']/kbtu_conversion_factor

In [0]:
del test_final['total_area']
del test_final['air_temperature']
del test_final['cloud_coverage']
del test_final['dew_temperature']
del test_final['precip_depth_1_hr']
del test_final['sea_level_pressure']
del test_final['wind_direction']
del test_final['wind_speed']
del test_final['weekday']
del test_final['hour']
del test_final['month']
del test_final['building_age']
del test_final['primary_use_Education']
del test_final['primary_use_Entertainment/public assembly']
del test_final['primary_use_Food sales and service']
del test_final['primary_use_Healthcare']
del test_final['primary_use_Lodging/residential']
del test_final['primary_use_Manufacturing/industrial']
del test_final['primary_use_Office']
del test_final['primary_use_Other']
del test_final['primary_use_Parking']
del test_final['primary_use_Public']
del test_final['primary_use_Religious worship']
del test_final['primary_use_Retail']
#del test_final['primary_use_Services']
del test_final['primary_use_Technology/science']
del test_final['primary_use_Utility']
del test_final['primary_use_Warehouse/storage']


In [0]:
del test_final['primary_use_Public services']

In [106]:
test_final.head()

,row_id,building_id,meter,timestamp,site_id,meter_reading
0,0,0,0,2017-01-01 00:00:00,0,12.188745
1,129,0,0,2017-01-01 01:00:00,0,11.786506
2,258,0,0,2017-01-01 02:00:00,0,10.432342
3,387,0,0,2017-01-01 03:00:00,0,10.569687
4,516,0,0,2017-01-01 04:00:00,0,12.495078


In [0]:
test_final.loc[(test_final['building_id'].isin(site_zero_buildings)) & (test_final['meter'] == 0),'meter_reading'] = test_final.loc[(test_final['building_id'].isin(site_zero_buildings)) & (test_final['meter'] == 0),'meter_reading']/kbtu_conversion_factor

In [0]:
#test_final.to_csv("train_final.csv",index=False)

In [0]:
submission_df = test_final[['row_id','meter_reading']]
submission_df = submission_df.sort_values(by=["row_id"])
#test_final.to_csv("submission.csv",index=False)

In [0]:
submission_df.to_csv("submission.csv",index=False)

In [0]:
#test_final.loc[(test_final['building_id'].isin(site_zero_buildings)) & (test_final['meter'] == 0),'meter_reading'] = test_final.loc[(test_final['building_id'].isin(site_zero_buildings)) & (test_final['meter'] == 0),'meter_reading']/kbtu_conversion_factor

In [0]:
store = pd.read_csv('/content/submission.csv')

In [119]:
store.shape

(41697600, 2)

In [0]:
#Finally got the submission data with exact rows